In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f

from pyspark.sql.types import *
import requests,json,time

In [0]:
#read the data from bronze
bronze_df=spark.readStream.format("delta").load("dbfs:/api_jan_bronze")

In [0]:
bronze_df1=bronze_df.withColumn("results", explode("results"))\
            .withColumn("Date", date_format(col("curr_timestamp"), "yyyy-MM-dd"))\
            .withColumn("Time", date_format(col("curr_timestamp"), "HH:mm:ss"))\
            .withColumn("Silver_timestamp",current_timestamp())\
            .select("nationality","results.user.gender","results.user.email","results.user.password","results.user.dob","results.user.phone","results.user.cell","results.user.registered","results.user.salt","results.user.sha256","results.user.username","results.user.sha1","results.user.md5","seed","version","results.user.name.*","results.user.location.*","results.user.picture.*","Date","Time","Silver_timestamp","curr_timestamp")

In [0]:
bronze_df2=bronze_df1.withColumn('username', regexp_replace('username', '[^a-zA-Z]', ''))\
    .withColumnRenamed('username',"nameoftheuser")

In [0]:
#create the another dataframe
dataframe2= spark.read.format("org.apache.spark.sql.avro.AvroFileFormat").load("dbfs:/FileStore/avro/projectsample.avro")

In [0]:
from pyspark.sql.functions import broadcast

In [0]:
#apply broadcast join
bronze_df3=bronze_df2.join(broadcast(dataframe2),bronze_df2["nameoftheuser"]==dataframe2["username"],"inner")

In [0]:
silver_df=bronze_df3.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "checkpoint_jan29") \
     .start("dbfs:/new_silver")